# データの結合

分析用のデータを作る際に、複数のデータソースを組み合わる場合があります。
例えば、1 年毎のデータに分かれているものを複数年分のデータに結合することや、アンケートデータと学校の成績のデータを同じ個人で結合したりします。

`pandas` では [`concat`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) 関数と [`merge`](https://pandas.pydata.org/docs/reference/api/pandas.merge.html) 関数などを用いてデータの結合をします。

データの長さ (インデックス) を増やすような結合を縦方向に結合すると呼んだりします。
縦の結合には、`concat` 関数を用います。

`concat` 関数は結合したい `Series` または `DataFrame` のリストを引数に入れ、`axis` キーワードで結合の方向を指定します。
デフォルトでは縦方向です。


In [1]:
import pandas as pd

In [2]:
df1 = pd.DataFrame({"A": [1, 2, 3], "B": [4, 5, 6]})
print(df1)
print("-" * 20)
df2 = pd.DataFrame({"A": [7, 8, 9], "C": [10, 11, 12]})
print(df2)
print("-" * 20)
print(pd.concat([df1, df2])) # 縦方向の結合

   A  B
0  1  4
1  2  5
2  3  6
--------------------
   A   C
0  7  10
1  8  11
2  9  12
--------------------
   A    B     C
0  1  4.0   NaN
1  2  5.0   NaN
2  3  6.0   NaN
0  7  NaN  10.0
1  8  NaN  11.0
2  9  NaN  12.0


`df1` の下に `df2` がつけ足されていることがわかります。
また、`df1` にないカラム `C`、`df2` にないカラム `B` はそれぞれ、`NaN` が足されています。

なお、縦方向に結合した場合、インデックスが重複する場合があります。
インデックスを新たに振りなおすには、`reset_index` メソッドを用います。

データにカラムを足すような結合は横方向の結合と呼びます。
`concat` でも `axis = "columns"` と指定することでこれが可能です。
インデックスが同じもの同士でデータが結合されます。

しかし、以下のように、カラムの重複が起こることもあります。

In [3]:
pd.concat([df1, df2], axis="columns") # 横方向の結合

,A,B,A,C
0,1,4,7,10
1,2,5,8,11
2,3,6,9,12


横方向の結合では、基本的に `merge` 関数または `join` メソッドを用います。
`merge` 関数には、結合する 2 つのデータを引数に入れます。
このとき、1 番目の引数に入れたものを左、2 番目の引数に入れたものを右として考えます。

`merge` では結合の種類、結合に使う変数 (キー) を指定します。
指定したキーの値が同じもの同士が、データとして結合されます。
キーのカラム名が同じ場合、`on` キーワードで指定します。
異なる場合は、それぞれ `left_on`, `right_on` キーワードで左、右のデータのキーとなるカラム名を指定します。

以下のデータを見ると、`df1` では `key` というカラムでは `A`, `B`, `C` があり、`df2` では `B`, `C`, `D` があり、一部異なっていることが分かります。

In [ ]:
df1 = pd.DataFrame({"key": ["A", "B", "C"], "value1": [1, 2, 3]})
print(df1)
print("-" * 20)
df2 = pd.DataFrame({"key": ["B", "C", "D"], "value2": [4, 5, 6]})
print(df2)
print("-" * 20)

In [ ]:


```py
# サンプルデータの作成

df1
  key  value1
0   A      1
1   B      2
2   C      3


df2
  key  value2
0   B      4
1   C      5
2   D      6
```

結合には 4 種類あり、内部結合、外部結合、左結合、右結合があります。
`merge` 関数では `how` キーワードで結合の種類を指定します。
デフォルトでは内部結合が選択されます。

内部 (inner) 結合は、キーが合致したもののみを結果として返します。
したがって、両者のキーで合致しない、`A`, `D` は除外されます。

```py
pd.merge(df1, df2, on = "key") # 内部結合
  key  value1  value2
0   B       2       4
1   C       3       5
```

外部 (outer) 結合は、キーが合致しないものも結果として返します。
したがって、両者のキーで合致しない `A`, `D` も結果に含みますが、`A`, `D` では合致しない部分のデータは欠損値になります。

```py
pd.merge(df1, df2, on = "key", how = "outer") # 外部結合
  key  value1  value2
0   A     1.0     NaN
1   B     2.0     4.0
2   C     3.0     5.0
3   D     NaN     6.0
```

左 (left) 結合は、左のデータを全て残し、キーが合致しないものも結果として返します。
したがって、左のキーに合致しない `D` は除外されますが、右のキーに合致しない `A` は残ります。

```py
pd.merge(df1, df2, on = "key", how = "left")
  key  value1  value2
0   A       1     NaN
1   B       2     4.0
2   C       3     5.0
```

右 (right) 結合は左結合の逆です。

```py
pd.merge(df1, df2, on = "key", how = "right")
  key  value1  value2
0   B     2.0       4
1   C     3.0       5
2   D     NaN       6
```